<a href="https://colab.research.google.com/github/DJCordhose/buch-machine-learning-notebooks/blob/master/kap10-function-calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Function Calling

* zum Ausführen ist ein OpenAI-Account notwendig: https://platform.openai.com/signup
* hier https://platform.openai.com/api-keys den eigenen Key erzeugen und eintragen
* Kosten
  * https://openai.com/pricing
  * https://platform.openai.com/usage
* Installation und Quickstart: https://platform.openai.com/docs/quickstart?context=python  

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules # True if running in Google Colab.
IN_COLAB

True

In [2]:
if IN_COLAB:
    !pip install -q openai tiktoken
    !pip install -q umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 540.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.8 MB/s eta 0:00:00


# OpenAI Client

In [3]:
# TODO: hier https://platform.openai.com/api-keys den eigenen Key erzeugen und eintragen
OPENAI_API_KEY = ''

assert OPENAI_API_KEY != '...', "Bitte tragen Sie Ihren OpenAI API Key ein!"

In [4]:
from openai import OpenAI

client = OpenAI(
  api_key=OPENAI_API_KEY
)
models = client.models.list()

# Function Calling

* https://platform.openai.com/docs/guides/function-calling
* https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models

In [5]:
anfrage = "Was ist das durschschnittliche Gehalt aller Mitarbeiter?"

In [16]:
# model="gpt-4o"
model="gpt-3.5-turbo"

tools = [
    {
        "type": "function",
        "function": {
            "name": "list_employee_ids",
            "description": "Enumerate the integer id of all employees",
            "parameters": {
                "type": "object",
                "properties": {
                }
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_employee_detail",
            # "description": "Gets the employee's details including salary and job title based on their id",
            "description": "Gets the employee's details",
            "parameters": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "integer",
                        "description": "The id of the employee like returned by list_employee_ids",
                    }
                },
                "required": ["id"],
            },
        },
    }

]

def user_input(messages, content = None):
  if content:
    message = {"role": "user", "content": content}
    messages.append(message)
  response = client.chat.completions.create(
      model=model,
      messages=messages,
      tools=tools,
  )
  response_message = response.choices[0].message
  messages.append(response_message)
  return response_message

In [17]:
employees = [
    {
        "name": "Oliver Zeigermann",
        "position": "CTO",
        "salary": 200_000
    },
    {
        "name": "Chi Nhan Nguyen",
        "position": "CEO",
        "salary": 250_000
    }
]

def get_employee_detail(id):
  print(id)
  if int(id) <= len(employees):
    return employees[id -1]
  return None

def list_employee_ids():
  return list(iter(range(1, len(employees) + 1)))

In [18]:
import json

def execute_tool_call(tool_call):
    function_name = tool_call.function.name
    function_to_call = globals()[function_name]
    function_args = json.loads(tool_call.function.arguments)

    function_response = function_to_call(**function_args)
    print(f"Calling {function_name} with {function_args}, getting {function_response}")
    return {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": json.dumps(function_response),
    }

def execute_tool_calls(messages, tool_calls):
  for tool_call in tool_calls:
    message = execute_tool_call(tool_call)
    messages.append(message)

In [19]:
messages = []
response_message = user_input(messages, anfrage)
messages

[{'role': 'user',
  'content': 'Was ist das durschschnittliche Gehalt aller Mitarbeiter?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ec47Abb8OnNuCO72SLS6RstZ', function=Function(arguments='{}', name='list_employee_ids'), type='function')])]

In [20]:
tool_calls = response_message.tool_calls
assert tool_calls

In [21]:
execute_tool_calls(messages, tool_calls)
messages

Calling list_employee_ids with {}, getting [1, 2]


[{'role': 'user',
  'content': 'Was ist das durschschnittliche Gehalt aller Mitarbeiter?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ec47Abb8OnNuCO72SLS6RstZ', function=Function(arguments='{}', name='list_employee_ids'), type='function')]),
 {'tool_call_id': 'call_ec47Abb8OnNuCO72SLS6RstZ',
  'role': 'tool',
  'name': 'list_employee_ids',
  'content': '[1, 2]'}]

In [22]:
response_message = user_input(messages)
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ec47Abb8OnNuCO72SLS6RstZ', function=Function(arguments='{"id": 1}', name='get_employee_detail'), type='function'), ChatCompletionMessageToolCall(id='call_IE3olMb9FC2Sfe5KZX6VoC5p', function=Function(arguments='{"id": 2}', name='get_employee_detail'), type='function')])

In [23]:
tool_calls = response_message.tool_calls
assert tool_calls
tool_calls

[ChatCompletionMessageToolCall(id='call_ec47Abb8OnNuCO72SLS6RstZ', function=Function(arguments='{"id": 1}', name='get_employee_detail'), type='function'),
 ChatCompletionMessageToolCall(id='call_IE3olMb9FC2Sfe5KZX6VoC5p', function=Function(arguments='{"id": 2}', name='get_employee_detail'), type='function')]

In [24]:
execute_tool_calls(messages, tool_calls)
messages

1
Calling get_employee_detail with {'id': 1}, getting {'name': 'Oliver Zeigermann', 'position': 'CTO', 'salary': 200000}
2
Calling get_employee_detail with {'id': 2}, getting {'name': 'Chi Nhan Nguyen', 'position': 'CEO', 'salary': 250000}


[{'role': 'user',
  'content': 'Was ist das durschschnittliche Gehalt aller Mitarbeiter?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ec47Abb8OnNuCO72SLS6RstZ', function=Function(arguments='{}', name='list_employee_ids'), type='function')]),
 {'tool_call_id': 'call_ec47Abb8OnNuCO72SLS6RstZ',
  'role': 'tool',
  'name': 'list_employee_ids',
  'content': '[1, 2]'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ec47Abb8OnNuCO72SLS6RstZ', function=Function(arguments='{"id": 1}', name='get_employee_detail'), type='function'), ChatCompletionMessageToolCall(id='call_IE3olMb9FC2Sfe5KZX6VoC5p', function=Function(arguments='{"id": 2}', name='get_employee_detail'), type='function')]),
 {'tool_call_id': 'call_ec47Abb8OnNuCO72SLS6RstZ',
  'role': 'tool',
  'name': 'get_employee_detail',
  'content': '{"name": "Oliver Zeigermann", "positi

In [25]:
response_message = user_input(messages)
antwort = response_message.content
antwort

'Die Gehälter der Mitarbeiter sind wie folgt:\n\n1. Oliver Zeigermann (CTO): 200.000 EUR\n2. Chi Nhan Nguyen (CEO): 250.000 EUR\n\nDer Durchschnitt der Gehälter beträgt:\n\\[ \\frac{200000 + 250000}{2} = \\frac{450000}{2} = 225000 \\]\n\nDas durchschnittliche Gehalt aller Mitarbeiter beträgt daher 225.000 EUR.'